In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value

        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)

        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)

        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output


class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x


class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2).to(src.device)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3).to(tgt.device)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(tgt.device)
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

# Pipeline for transformer


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import json
import math
from collections import Counter


In [4]:
# =============================
# Vocabulary and Tokenization
# =============================

# def build_vocab(file, min_freq=2, special_tokens=["<pad>", "<unk>", "<sos>", "<eos>"]):
#     with open(file, encoding="utf-8") as f:
#         words = Counter(word for line in f for word in line.strip().split())
#     vocab = {token: idx for idx, token in enumerate(special_tokens)}
#     for word, freq in words.items():
#         if freq >= min_freq and word not in vocab:
#             vocab[word] = len(vocab)
#     return vocab

# def sentence_to_ids(sentence, vocab, max_len=50):
#     tokens = sentence.split()
#     ids = [vocab.get("<sos>")] + [vocab.get(token, vocab["<unk>"]) for token in tokens[:max_len]] + [vocab.get("<eos>")]
#     return ids + [vocab.get("<pad>")] * (max_len - len(ids))

In [6]:
from collections import Counter

def build_vocab(file, min_freq=2, special_tokens=["<pad>", "<unk>", "<sos>", "<eos>"]):
    with open(file, encoding="utf-8") as f:
        words = Counter(word for line in f for word in line.strip().split())
    vocab = {token: idx for idx, token in enumerate(special_tokens)}
    for word, freq in words.items():
        if freq >= min_freq and word not in vocab:
            vocab[word] = len(vocab)
    return vocab

def sentence_to_ids(sentence, vocab, max_len=50):
    tokens = sentence.split()
    ids = [vocab.get("<sos>")] + [vocab.get(token, vocab["<unk>"]) for token in tokens[:max_len]] + [vocab.get("<eos>")]
    return ids + [vocab.get("<pad>")] * (max_len - len(ids))

# Build vocabularies
bangla_vocab = build_vocab("/teamspace/studios/this_studio/Attention-Is-All-You-Need/data/bn_data.txt")
english_vocab = build_vocab("/teamspace/studios/this_studio/Attention-Is-All-You-Need/data/en_data.txt")

# Example sentences
bangla_sentence = "জেনেটের হাঁসগুলি প্রতিদিন 16 টি করে ডিম পাড়ে।"
english_sentence = "Janet's ducks lay 16 eggs every day."

# Convert sentences to ids
bangla_ids = sentence_to_ids(bangla_sentence, bangla_vocab)
english_ids = sentence_to_ids(english_sentence, english_vocab)

print("Bangla IDs:", bangla_ids)
print("English IDs:", english_ids)

Bangla IDs: [2, 1, 1, 4, 755, 806, 6, 7, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
English IDs: [2, 1, 4, 1, 5, 6, 14, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset

class TranslationDataset(Dataset):
    def __init__(self, src_file, tgt_file, src_vocab, tgt_vocab, max_len=50):
        with open(src_file, encoding="utf-8") as f:
            self.src_sentences = f.readlines()
        with open(tgt_file, encoding="utf-8") as f:
            self.tgt_sentences = f.readlines()
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.src_sentences)

    def __getitem__(self, idx):
        src_ids = sentence_to_ids(self.src_sentences[idx].strip(), self.src_vocab, self.max_len)
        tgt_ids = sentence_to_ids(self.tgt_sentences[idx].strip(), self.tgt_vocab, self.max_len)
        return torch.tensor(src_ids), torch.tensor(tgt_ids)

In [8]:
# Define the custom collate function
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src, tgt in batch:
        src_batch.append(src)
        tgt_batch.append(tgt)
    
    src_batch = pad_sequence(src_batch, batch_first=True, padding_value=0)
    tgt_batch = pad_sequence(tgt_batch, batch_first=True, padding_value=0)
    
    return src_batch, tgt_batch

In [9]:
# =============================
# Training and Evaluation
# =============================

def train_model(model, dataloader, optimizer, criterion, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for i, (src, tgt) in enumerate(dataloader):
            src, tgt = src.to(device), tgt.to(device)
            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]

            optimizer.zero_grad()
            output = model(src, tgt_input)

            # Fix: Use reshape instead of view
            loss = criterion(output.reshape(-1, output.shape[-1]), tgt_output.reshape(-1))

            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(dataloader)}")



def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)
            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]

            output = model(src, tgt_input)
            loss = criterion(output.view(-1, output.shape[-1]), tgt_output.view(-1))
            total_loss += loss.item()
    return total_loss / len(dataloader)

def translate_sentence(model, src_sentence, src_vocab, tgt_vocab, max_len=50, device="cpu"):
    model.eval()
    src_ids = sentence_to_ids(src_sentence, src_vocab, max_len)
    src_tensor = torch.tensor(src_ids).unsqueeze(0).to(device)
    tgt_ids = [tgt_vocab["<sos>"]]

    for _ in range(max_len):
        tgt_tensor = torch.tensor(tgt_ids).unsqueeze(0).to(device)
        output = model(src_tensor, tgt_tensor)
        next_token = output.argmax(2)[:, -1].item()
        tgt_ids.append(next_token)
        if next_token == tgt_vocab["<eos>"]:
            break

    return " ".join([key for key, val in tgt_vocab.items() if val in tgt_ids])


In [27]:
# =============================
# Main Execution
# =============================

if __name__ == "__main__":
    # Build vocabularies
    src_vocab = build_vocab("/teamspace/studios/this_studio/Attention-Is-All-You-Need/data/bn_data.txt")
    tgt_vocab = build_vocab("/teamspace/studios/this_studio/Attention-Is-All-You-Need/data/en_data.txt")

    # Save vocabularies (optional)
    with open("src_vocab.json", "w", encoding="utf-8") as f:
        json.dump(src_vocab, f, indent=4)
    with open("tgt_vocab.json", "w", encoding="utf-8") as f:
        json.dump(tgt_vocab, f, indent=4)

    # Create dataset and dataloader
    dataset = TranslationDataset("/teamspace/studios/this_studio/Attention-Is-All-You-Need/data/bn_data.txt", "/teamspace/studios/this_studio/Attention-Is-All-You-Need/data/en_data.txt", src_vocab, tgt_vocab)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [24]:
import torch
import torch.nn as nn

class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, nhead, num_encoder_layers, 
                 num_decoder_layers, dim_feedforward, max_len=5000):
        super(TransformerModel, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_len)
        self.transformer = nn.Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward)
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(0.1)

    def generate_mask(self, src, tgt):
        src_seq_len = src.shape[1]
        tgt_seq_len = tgt.shape[1]
        src_mask = self.transformer.generate_square_subsequent_mask(src_seq_len).to(src.device)
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt_seq_len).to(tgt.device)
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))
        enc_output = self.transformer.encoder(src_embedded, src_mask)
        dec_output = self.transformer.decoder(tgt_embedded, enc_output, tgt_mask)
        output = self.fc_out(dec_output)
        return output

In [28]:
# Initialize model
model = TransformerModel(
    src_vocab_size=len(src_vocab),
    tgt_vocab_size=len(tgt_vocab),
    d_model=512,
    nhead=8,
    num_encoder_layers=6,
    num_decoder_layers=6,
    dim_feedforward=2048,
)
## Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=src_vocab["<pad>"])

# Train model
device = "cuda" if torch.cuda.is_available() else "cpu"

In [29]:
train_model(model, dataloader, optimizer, criterion, num_epochs=2, device=device)

# Save model
torch.save(model.state_dict(), "transformer_translation.pth")

RuntimeError: The shape of the 2D attn_mask is torch.Size([52, 52]), but should be (32, 32).

In [ ]:






# # Example loop to access the DataLoader
# for i, (src, tgt) in enumerate(dataloader):
#     print(f"Batch {i}:")
#     print("Source:", src)
#     print("Target:", tgt)

Batch 0:
Source: tensor([[   2,    1,    1,  ...,    0,    0,    0],
        [   2,    1,   84,  ...,    0,    0,    0],
        [   2, 1148,  221,  ...,    0,    0,    0],
        ...,
        [   2, 1272,    1,  ...,    0,    0,    0],
        [   2,   99,   72,  ...,    0,    0,    0],
        [   2,  824,  456,  ...,    0,    0,    0]])
Target: tensor([[   2,  185,  930,  ...,    0,    0,    0],
        [   2,    1,  534,  ...,    0,    0,    0],
        [   2, 1033,   16,  ...,    0,    0,    0],
        ...,
        [   2, 1145,  477,  ...,    0,    0,    0],
        [   2,  100,  101,  ...,    0,    0,    0],
        [   2,  756,  116,  ...,    0,    0,    0]])
Batch 1:
Source: tensor([[   2,    1,  526,  ...,    0,    0,    0],
        [   2,    1,   15,  ...,    0,    0,    0],
        [   2,  484,  857,  ...,    0,    0,    0],
        ...,
        [   2, 1266, 1267,  ...,   98,    3,    0],
        [   2,  620,  112,  ...,    0,    0,    0],
        [   2,    1,   38,  ..., 

# Testing Model

In [ ]:
# Re-initialize the model architecture
#model = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Load the trained weights
model.load_state_dict(torch.load('transformer_translation.pth'))
model.eval()  # Set the model to evaluation mode


In [ ]:
test_sentences = [
    "তাঁর বাবার নাম হাউস সরদার এবং মায়ের নাম জরিমন নেছা",
    "তাঁর স্ত্রীর নাম আনোয়ারা খাতুন ওরফে আনারজান"
]


In [ ]:
def preprocess_input(sentence, src_vocab):
    # Tokenize the sentence (you may use a tokenizer like word_tokenize or simple split)
    sentence_tokens = sentence.split()  # Basic split (you can improve this with a more sophisticated tokenizer)
    # Convert tokens to their corresponding word IDs using the vocabulary
    sentence_ids = [src_vocab.get(token, src_vocab["<unk>"]) for token in sentence_tokens]
    return sentence_ids

# Preprocess the test sentences
test_input_ids = [preprocess_input(sentence, src_vocab) for sentence in test_sentences]


In [ ]:
# Create reverse vocab for tgt_vocab (so we can decode from word IDs back to words)
reverse_tgt_vocab = {v: k for k, v in tgt_vocab.items()}


In [ ]:
def translate_sentence(src, model, max_length=50):
    # Convert the input sentence to a tensor (batch_size = 1)
    src_tensor = torch.tensor(src).unsqueeze(0).to(device)

    # Create mask for the source sequence
    src_mask = (src_tensor != src_vocab["<pad>"]).unsqueeze(1).unsqueeze(2).to(device)

    # Decoder input (start with <sos> token)
    tgt_input = torch.tensor([tgt_vocab["<sos>"]]).unsqueeze(0).to(device)

    # Store the output sequence
    translated_sentence = []

    # Iterate to generate the translated sentence token by token
    for _ in range(max_length):
        output = model(src_tensor, tgt_input)  # Forward pass through the model
        output_token = output.argmax(dim=-1)[:, -1].item()  # Get the predicted token (last in the sequence)

        if output_token == tgt_vocab["<eos>"]:
            break  # Stop when <eos> is generated

        translated_sentence.append(output_token)

        # Update tgt_input for the next token (append the predicted token)
        tgt_input = torch.cat((tgt_input, torch.tensor([[output_token]]).to(device)), dim=-1)

    # Convert output tokens back to words
    translated_words = [reverse_tgt_vocab[token] for token in translated_sentence]
    return " ".join(translated_words)

# Translate the test sentences
for sentence in test_sentences:
    input_ids = preprocess_input(sentence, src_vocab)
    translation = translate_sentence(input_ids, model)
    print(f"Bangla: {sentence}")
    print(f"English: {translation}")


In [ ]:
samples = [
    "এলোমেলোভাবে তুলে নেওয়া প্রথম সঠিক উত্তরদাতাকেই বিজয়ী ঘোষণা করা হবে।",
    "আমার সোনার বাংলা, আমি তোমায় ভালোবাস।"
]

In [ ]:
for s in samples:
    input_ids = preprocess_input(s, src_vocab)
    translation = translate_sentence(input_ids, model)
    print(f"Bangla: {s}")
    print(f"English: {translation}")